In [1]:
import re
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\Admin\Documents\Học tập\CRAWL WEB\thegioididong_laptop_full.csv")


In [2]:
# Xử lý cột "screen_size" để chỉ giữ lại số và chuyển sang kiểu float
df["screen_size"] = (
    df["screen_size"]
    .astype(str)
    .str.lower()
    .str.replace('"', '', regex=False)
    .str.replace('inch', '', regex=False)
    .str.strip()
    .str.extract(r'(\d+\.?\d*)')   # chỉ lấy số đầu tiên
)

df["screen_size"] = pd.to_numeric(df["screen_size"], errors="coerce")


In [3]:
# Xử lý cột "ram" để chỉ giữ lại số và chuyển sang kiểu int
df["ram"] = (
    df["ram"]
    .astype(str)
    .str.extract(r'(\d+)')
)

df["ram"] = pd.to_numeric(df["ram"], errors="coerce").astype("Int64")


In [4]:
# Xử lý cột "storage" để chuẩn hóa về GB

def clean_storage(value):
    if pd.isna(value):
        return np.nan

    s = str(value).upper()

    # Tìm số + đơn vị đầu tiên (GB hoặc TB)
    match = re.search(r'(\d+\.?\d*)\s*(TB|GB)', s)
    if not match:
        return np.nan

    size = float(match.group(1))
    unit = match.group(2)

    if unit == "TB":
        return int(size * 1024)
    else:  # GB
        return int(size)

df["storage"] = df["storage"].apply(clean_storage).astype("Int64")


In [5]:
# Xử lý cột "battery" để chỉ giữ lại số và chuyển sang kiểu float
def extract_battery_wh(value):
    if pd.isna(value):
        return np.nan
    
    s = str(value).upper()
    
    # Tìm số đứng trước WH
    match = re.search(r'(\d+\.?\d*)\s*WH', s)
    
    if match:
        return float(match.group(1))
    
    return np.nan

df["battery_wh"] = df["battery"].apply(extract_battery_wh)

df.drop(columns=["battery"], inplace=True)

In [6]:
# Xử lý cột "price" để chỉ giữ lại số và chuyển sang kiểu int
df["price"] = (
    df["price"]
    .astype(str)
    .str.replace(r"[^\d]", "", regex=True)  # chỉ giữ lại số
)

df["price"] = pd.to_numeric(df["price"], errors="coerce").astype("Int64")

In [7]:
# Xử lý cột "rating" để chỉ giữ lại số và chuyển sang kiểu float
df["rating"] = (
    df["rating"]
    .astype(str)
    .str.extract(r'(\d+\.?\d*)')
    .astype(float)
)


In [8]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439 entries, 0 to 438
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               439 non-null    object 
 1   cpu                439 non-null    object 
 2   ram                438 non-null    Int64  
 3   gpu                438 non-null    object 
 4   storage            438 non-null    Int64  
 5   screen_size        438 non-null    float64
 6   screen_resolution  438 non-null    object 
 7   screen_panel       345 non-null    object 
 8   color              19 non-null     object 
 9   price              428 non-null    Int64  
 10  rating             380 non-null    float64
 11  review_text        91 non-null     object 
 12  battery_wh         437 non-null    float64
dtypes: Int64(3), float64(3), object(7)
memory usage: 46.0+ KB


,name,cpu,ram,gpu,storage,screen_size,screen_resolution,screen_panel,color,price,rating,review_text,battery_wh
0,Laptop MacBook Air 13 inch M4 16GB/256GB,Apple M4,16,Card tích hợp - 8 nhân GPU,256,13.6,Liquid Retina (2560 x 1664),IPS,đen,23990000,4.9,Mình mua gần 1 tháng. mà mở 1 tay ko đc. bảng ...,53.8
1,"Laptop HP 15 fc0655AU - C81NGPA (R5 7430U, 16G...",AMD Ryzen 5 - 7430U,16,Card tích hợp - AMD Radeon Graphics,512,15.6,Full HD (1920 x 1080),Hãng không công bố,NaN,14990000,5.0,NaN,41.0
2,Laptop Asus Vivobook Go E1504FA - BQ1160W (R5 ...,AMD Ryzen 5 - 7520U,16,Card tích hợp - AMD Radeon Graphics,512,15.6,Full HD (1920 x 1080),IPS,NaN,14690000,5.0,NaN,42.0
3,"Laptop Dell 15 DC15250 - DC5I5357W1 (i5 1334U,...",Intel Core i5 Raptor Lake - 1334U,16,Card tích hợp - Intel UHD Graphics,512,15.6,Full HD (1920 x 1080),IPS,NaN,17490000,5.0,NaN,41.0
4,Laptop Lenovo IdeaPad Slim 3 15IRH10 - 83K1000...,Intel Core i5 Raptor Lake - 13420H,16,Card tích hợp - Intel UHD Graphics,512,15.3,WUXGA (1920 x 1200),IPS,NaN,16690000,4.9,"Màn hình rộng, chip ổn, hành trình gõ phím tốt...",60.0


In [9]:
final_columns = [
    "name",
    "cpu",
    "ram",
    "gpu",
    "storage",
    "screen_size",
    "screen_resolution",
    "screen_panel",
    "battery_wh",
    "color",
    "price",
    "rating",
    "review_text"
]

df = df[final_columns]


In [10]:
df.to_csv(
    "laptop_data_cleaned.csv",
    index=False,
    encoding="utf-8-sig"
)
